# Assignment 7

Name 1: Sangeet Sagar<br/>
Student id 1: 7009050<br/>
Email 1: sasa00001@stud.uni-saarland.de<br/>


Name 2: Nikhil Paliwal<br/>
Student id 2: 7009915<br/>
Email 2: nipa00002@stud.uni-saarland.de<br/> 

**Instructions:** Read each question carefully. <br/>
Make sure you appropriately comment your code wherever required. Your final submission should contain the completed Notebook and the Python files for exercises 1 and 2. There is no need to submit the data files. <br/>
Upload the zipped folder in Teams. Make sure to click on "Turn-in" after your upload your submission, otherwise the assignment will not be considered as submitted. Only one from the group should make the submisssion.

---

In [ ]:

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import matplotlib.pyplot as plt
import seaborn
import matplotlib as mpl
seaborn.set()
mpl.rcParams['figure.dpi'] = 120

## Exercise 1: Count-Trees (5 points)

Your task is to implement a count-tree with variable maximum history. It is a memory-efficient way to store n-gram counts which can also be used to create an intuitive back-off after the tree is pruned.

The tree object should support the following four operations:
- Increment a count of a specific n-gram (even if not present)
- Retrieve counts given the history (variable length)
- Retrieve the conditional probability of a word given the history. (Proportion count between branches)
- Pruning all nodes with counts less or equal to $k$

**1.1 (2 points)**

Make sure your implementation is correct by passing the asserts in the first code cell.

**1.2 (1 point)**

The next cell will incrementally add a quad-gram to the tree. Plot the perplexity of trigram language model (induced by this count tree) against the number of added n-grams. Comment on the curve shape. Smooth this language model with a zerogram distribution using a linear combination ($0.75\times p_4 + 0.25\times p_0$).

**1.3 (1 point)**

For the given range of thresholds, prune your tree and see how the threshold affects the performance. Plot the results (perplexity vs. threshold).

**1.4 (1 point)**

1. If you first prune with threshold $k_1$ and get tree $t_1$, then prune with $k_2$ and get $t_2$ what will be the relationship between $t_1$ and $t_2$ if $k_1 \ge k_2$? (0.25 points)
2. What is the memory benefit of count trees, in comparison to storing the counts as a dictionary `{n-gram:freq}`? (0.25 points)
3. If we pruned the tree so that only the first level is preserved, what distribution could we model with this tree? (0.25 points)
4. Pruning the count tree is said to be a dynamic way of smoothing the language model. Elaborate on how this smoothing happens. (0.25 points)

In [1]:
from importlib import reload
import exercise_1
exercise_1 = reload(exercise_1)

tree = exercise_1.CountTree(n=4)

assert tree.get("") == 0
tree.add("ABCE")
tree.add("ABCD")
tree.add("ABCD")
tree.add("QBCD")
tree.add("QQCD")
tree.add("BCDA")
tree.add("1234")
tree.add("1234")
tree.add("1234")
tree.add("1234")
tree.add("1234")
tree.add("5634")
assert tree.get("ABCD") == 2
assert tree.get("ABCX") == 0
assert tree.get("BCD") == 3
assert tree.get("D") == 4
assert tree.get("CD") == 4
assert tree.get("1234") == 5
assert tree.get("5634") == 1

In [ ]:
tree.prune(4)
assert tree.get("ABCD") == 4
assert tree.get("XXCD") == 4
assert tree.get("D") == 4
assert tree.get("1234") == 5
assert tree.get("5634") == 1

In [ ]:
import matplotlib.pyplot as plt

plot_x = []
plot_y = []

ngrams = []
tree = exercise_1.CountTree(n=4)
with open("data/alice_in_wonderland.txt", "r") as f:
  tokens = f.read().lower().split()
  for i in range(len(tokens)-4):
    ngrams.append(tokens[i:i+4])

vocab = set(tokens)
for i,ngram in enumerate(ngrams):
  tree.add(ngram)
  if i % 1000 == 0:
    plot_x.append(i)
    plot_y.append(tree.perplexity(ngrams, vocab))

plt.plot(plot_x, plot_y)

In [ ]:
plot_x = []
plot_y = []

for threshold in [1,2,3,4,5,10,25,50,75,100]:
  tree.prune(threshold)
  plot_x.append(threshold)
  plot_y.append(tree.perplexity(ngrams, vocab))

print(plot_x)
print(plot_y)

plt.plot(plot_x, plot_y)

## Exercise 2: Kneser-Ney Smoothing (5 points)

This exercise aims to provide a basic understanding of Kneser-Ney Smoothing. Kneser-Ney Smoothing makes use of *continuation counts* of words for lower order n-grams, given as

\begin{equation}
C_{KN} = 
\begin{cases}
\text{count}(\bullet) & \text{for highest order} \\
\text{continuationcount}(\bullet) & \text{for lower orders}
\end{cases}
\end{equation}


For a trigram distribution, Kneser-Ney Smoothing is implemented using the following equations:

$$P_{KN}(w_3|w_1, w_2) = \frac{\max\{N(w_1 w_2 w_3)-d,0\}}{N(w_1 w_2)} + \lambda(w_1, w_2)P_{KN}(w_3|w_2)$$

$$P_{KN}(w_3|w_2) = \frac{\max\{N_{+}(\bullet w_2 w_3)-d,0\}}{N_{+}(\bullet w_2 \bullet)} + \lambda(w_2)P_{KN}(w_3)$$

\begin{equation}
P_{KN}(w_3) = \begin{cases}
\frac{N_{+}(\bullet w_3)}{N_{+}(\bullet \bullet)} & \text{if $w_3 \in$ V} \\
\frac{1}{V} & \text{otherwise}
\end{cases}
\end{equation}

$\lambda$ is used to normalise the discounted probability mass and is given by

$$\lambda(w_1, w_2) = \frac{d}{N(w_1 w_2)} \cdot N_{+}(w_1 w_2 \bullet)$$

$$\lambda(w_2) = \frac{d}{N(w_2)} \cdot N_{+}(w_2 \bullet)$$

**2.1 (4.5 points)**

* Your first task is to understand what these terms represent and fill it in the table below (4-5 words each).

* Create a trigram-level model on the given text, `alice_in_wonderland.txt`. Write your implementation in the file `exercise_2.py`. Preprocess the text by punctuation removal, lowercasing, and tokenisation. There is no need to split the data into train and test sets. (0.5 points)

In [2]:
from importlib import reload
import exercise_2
exercise_2 = reload(exercise_2)

file = open("data/alice_in_wonderland.txt", "r")
text = file.read()

# TODO: Preprocess text
tokens = exercise_2.preprocess(text)

* 
Write a simple class `KneserNey` in `exercise_2` that calculates the different parameters required for finding the trigram conditional probability. You may modify the function signature and add other functionality as required. <br/>
Now, consider the trigrams `"alice said nothing"` and `"alice said nichts"`. For these trigrams, estimate the values mentioned in the table given below and fill in the obtained results. The discounting parameter *d* = 0.75. (3 points)

In [3]:
KN_model = exercise_2.KneserNey(tokens, d=0.75, N=3)

t1 = "alice said nothing"
t2 = "alice said nichts"

# TODO
# Get the required parameters
KN_model.global_counter(tokens)

params_t1 = KN_model.get_params(t1)
print(params_t1)
print('\n')
params_t2 = KN_model.get_params(t2)
print(params_t2)

V=  2749
{'Nw1_w2_w3': 2, 'Nw1_w2': 11, 'Nplus_dot_w2_w3': 5, 'Nplus_dot_w2_dot': 319, 'Nplus_dot_w3': 23, 'Nplus_dot_dot': 14713, 'Nplus_w1_w2_dot': 6, 'Nplus_w2_dot': 56, 'lambda_w1_w2': 0.40909090909090906, 'lambda_w2': 0.09090909090909091}


V=  2749
{'Nw1_w2_w3': 0, 'Nw1_w2': 11, 'Nplus_dot_w2_w3': 0, 'Nplus_dot_w2_dot': 319, 'Nplus_dot_w3': 0, 'Nplus_dot_dot': 14713, 'Nplus_w1_w2_dot': 6, 'Nplus_w2_dot': 56, 'lambda_w1_w2': 0.40909090909090906, 'lambda_w2': 0.09090909090909091}


| Term in Kneser-Ney |   Value t1  | Value t2 | Description | 
|---|---|---| --- |
|$N(w_1w_2w_3)$|2  |0  |number of trigrams of the <br> combination $w_1 w_2 w_3$ |
|$N(w_1 w_2)$| 11 |11  | number of bigrams of the <br> combination $w_1 w_2$|
|$N_{+}( \bullet w_2 w_3)$|5  |0  |number of words that precede <br> $w_2 w_3$ at least once in the corpus |
|$N_{+}( \bullet w_2 \bullet)$|319  |319  |number of trigrams that precede $w_2$<br> and also has history $w_2$|
|$N_{+}( \bullet w_3)$|23  | 0 |unique number of words that precede <br> $w_3$ at least once in the corpus |
|$N_{+}( \bullet \bullet)$|14713  |14713  |all bigrams appearing <br> atleast once |
|$N_{+}(w_1 w_2 \bullet)$|6 |6 |number of words observed<br> after $w_1 w_2$ atleast once|
|$N_{+}(w_2 \bullet)$|56  | 56 | number of words observed<br> after $w_2$ atleast once|
|$\lambda(w_1 w_2)$|0.409 |0.409 |scaling factor for bigram $w_1 w_2$ |
|$\lambda(w_2)$|0.090 |0.090 | scaling factor for unigram $w_1$|

* Using the values obtained above, manually calculate $P_{KN}(w_3)$, $P_{KN}(w_3|w_2)$, and $P_{KN}(w_3|w_1, w_2)$ for the given trigrams. (1 point)

**Answers** <br>
\begin{align*}
P_{KN}(w_3=\text{nothing}) &= 23/14713 = 0.001563 \\
P_{KN}(w_3=\text{nothing}|w_2=\text{said}) &= \frac{max(5-0.75, 0)}{319} + 0.09090 \cdot 0.001563 = 0.01346496 \\
P_{KN}(w_3=\text{nothing}|w_1=\text{alice}, w_2=\text{said}) &= \frac{max(2-0.75, 0)}{11} + 0.40909 \cdot 0.01346496 = 0.1191447 \\
\\
P_{KN}(w3=\text{nichts}) &= 1/2749 = 0.000363 \\
P_{KN}(w_3=\text{nichts}|w_2=\text{said}) &= \frac{max(0-0.75, 0)}{319} + 0.09090 \cdot 0.000363 = 3.29e-05 \\
P_{KN}(w_3=\text{nichts}|w_1=\text{alice}, w_2=\text{said}) &= \frac{max(0-0.75, 0)}{11} + 0.40909 \cdot 3.29e-05 = 1.34e-05
\end{align*}


**2.2 (0.5 points)**

Take a look at this [video](https://www.youtube.com/watch?v=cbAxvpBFyNU) on Kneser-Ney smoothing by Dan Jurafksy. Make sure to undestand his *San Francisco* example. <br/>
How will Kneser-Ney Smoothing handle the following bigrams (answer in 3-4 sentences)? 

* Abu Dhabi

* Game Over

**Answer** <br>
In the bigram `Abu Dhabi`, the word `Dhabi` appears often but only after `Abu` while in `Start Over`, the word `Over` doesnt appear only after `Over` but also after many other possible words like `pull`, `talk`,`stop` etc. So, the unigram probablity should only be based on number of different words it follows. Therefore, the continuation probablity for `Game over` would be greater than `Abu Dhabi`. 

Also, the quantity $P_{KN}(w_i|w_{i-1})$ 
$$ \frac{C(w_i=\text{Dhabi},w_{i-1}=\text{Abu})}{C(w_i=\text{Abu})}  > \frac{C(w_i=\text{Over},w_{i-1}=\text{Game})}{C(w_i=\text{Game})} $$
Since counts of `Dhabi` would be almost same as counts of `Abu Dhabi` while counts of `Game` would be greater than `Game Over`

## Bonus (2 points)

For each of the smoothing techniques below,

1. Laplace/add-1 smoothing (0.3 points)
2. Add-$\alpha$ smoothing (0.3 points)
3. Linear interpolation (0.3 points)
4. Absolute discounting (0.3 points)
5. Good-Turing (0.3 points)
6. Kneser-Ney smoothing (0.3 point)

* Give the intuition behind it
* State at least one drawback and
* Explain how the ensuing smoothing technique accounts for this drawback.

You can do so in continuous text or in bullet points. Write 3-5 sentences for each technique. For Kneser-Ney smoothing, you should suggest *and explain* an improved version from the literature, e.g. [here](http://nrs.harvard.edu/urn-3:HUL.InstRepos:25104739) (this tutorial may also be helpful for the rest of the exercise).

Please note that while the points for this bonus exercise are the immediate motivation, your self-made comparison will be highly beneficial for the exam.

1. 
- In order to deal with the 0 probability of an n-gram, we pretend to see each word an extra number of times. So we add 1 to all counts
- It sometimes assigns too much probability mass from available ngrams in training data to ngrams absent in training data
- Since adding 1 to all counts leads to the probability mass getting assigned too much (to absent ngrams), how about we control this number. We can change to a float value less than 1.

2. 
- In add $\alpha$ smoothing, we fix the problem of 0 probability by adding $\alpha<1$ to each count. So the formula becomes:
$$ P_{+\alpha}(w_i|w_{i-1}) = \frac{C(w_{i-1},w_i)+\alpha}{C(w_{i-1})+\alpha \cdot V}$$
- Even if we optimize $\alpha$ to a suitable optimized value, this smoothing technique makes pretty bad predictions for word sequences.
- Solution? we can try a different approach. until now we have been using counts of unigrams and bigrams to compute the conditional probability of bigrams and trigrams respectively. We cant use e.g. trigrams if we have very low evidence of it. Idea? What if we use all of the unigrams, bigrams, and trigrams. Let's see this approach.

3. 
- Linear interpolation suggests that we use evidences from all of the unigrams, bigrams, and trigrams and multiply a factor ($\lambda_1$, $\lambda_2$, $\lambda_3$) to each of them to control their properties that are favorable for our language model.
- The optimal $\lambda_i$ depends on the context but we cant tune all lambdas separately. We will need to bucket them.
- 

4. 
- In absolute discounting  a constant value $ m$ is subtracted from each count. The effect of this is that the ngrams with the lowest counts are discounted relatively more than those with higher counts.
- The problem with this method is that if we haven't seen a bigram at all. We are going to reply only on the unigram probability.
- Good turing addresses this problem 

5. 
- The intitution behind Good-Turning estimation is to use the counts of ngrams we have seen once to estimate the counts of ngrams we have never seen.
$$
c^*_k = (c_k+1)\frac{N_{k+1}}{N_k}
$$
- when $k$ is very large i.e. its values is equal to maximum occurrence in the training corpus (for eg. $k$=3 and there are no items are that appear $k+1$ times), then there is a possible conflict with the defined formula ($N_{k+1}=0$). We also have to make sure that the probabilities are always normalized

6. 
- It is a modified version of absolute discounting. In this, we optimize the calculation of the lower-order n-gram probabilities in case the higher-order ngram was unseen in the corpus
- The given article suggests not to use the same $d$ discounting parameter for all non-zero counts. It writes that we can have three different $D_1$, $D_2$, $D_{3+}$ that can be applied to unigrams, bigrams, and trigrams respectively